In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load the model and video
model = YOLO(r'C:\Users\poory\Desktop\work\bonji\action detection\track\best.pt')
video_path = r'C:\Users\poory\Desktop\work\bonji\action detection\track\video\croped2.avi'
output_video_path = r'C:\Users\poory\Desktop\work\bonji\action detection\track\cropped_output.avi'

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
frame_height, frame_width = frame.shape[:2]

# Define output video settings (e.g., 640x480)
output_width, output_height = 640, 480
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # or 'MJPG', 'MP4V', etc., based on your format
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (output_width, output_height))

frame_skip_interval = 20
frame_count = 0

while ret:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Skip frames based on the frame_skip_interval
    if frame_count % frame_skip_interval != 0:
        continue

    # Perform tracking on the frame
    results = model.track(frame, persist=True)

    # Create an empty frame to add cropped persons
    cropped_frame = cv2.zeros((output_height, output_width, 3), dtype=np.uint8)

    # Track each person detected
    for detection in results[0].boxes:
        # Get bounding box coordinates
        x1, y1, x2, y2 = map(int, detection.xyxy[0])  # Convert to integers

        # Crop and resize the detected bounding box area
        cropped_person = frame[y1:y2, x1:x2]
        cropped_person_resized = cv2.resize(cropped_person, (output_width, output_height))

        # Overlay the resized crop on the `cropped_frame`
        cropped_frame = cropped_person_resized

    # Write the cropped frame to the output video
    out.write(cropped_frame)

    # Display the processed frame
    cv2.imshow('Cropped Frame', cropped_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
